# successful try od fiducial features

## tompkins do automatic segmentation

In [ ]:
import numpy as np
from scipy.signal import butter, filtfilt, find_peaks

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

def pan_tompkins(ecg_signal, fs=1000):
    # Bandpass filter the ECG signal
    ecg_signal_filtered = butter_bandpass_filter(ecg_signal, 5, 15, fs)

    # Differentiate the signal
    diff_signal = np.diff(ecg_signal_filtered)

    # Square the signal
    squared_signal = diff_signal ** 2

    # Integrate the signal
    integrated_signal = np.convolve(squared_signal, np.ones(fs // 8))[:len(ecg_signal)]
# ######
    # Find the QRS complex peaks using a threshold-based approach
    qrs_peaks, _ = find_peaks(integrated_signal, height=0.35*np.max(integrated_signal), distance=0.2*fs)

    # Calculate the fiducial features
    qrs_onsets = []
    qrs_offsets = []
    p_peaks = []
    t_peaks = []
    rr_interval_low = int(fs * 0.2)
    rr_interval_high = int(fs * 1)
    
    for qrs_peak in qrs_peaks:
        # Find the QRS onset
        # Find the QRS onset and offset
        qrs_onset = qrs_peak - int(rr_interval_low * 0.5)
        qrs_offset = qrs_peak + int(rr_interval_high * 0.5)
        qrs_onsets.append(qrs_onset)
        qrs_offsets.append(qrs_offset)

#         # Find the QRS onset
#         for i in range(qrs_peak, 0, -1):
#             if integrated_signal[i] < integrated_signal[qrs_peak] * 0.15:
#                 qrs_onsets.append(i)
#                 break
#         # Find the QRS offset
#         for i in range(qrs_peak, len(ecg_signal)):
#             if integrated_signal[i] < integrated_signal[qrs_peak] * 0.2:
#                 qrs_offsets.append(i)
#                 break
                
                
        # Find the P wave peak
        p_peak, _ = find_peaks(ecg_signal_filtered[qrs_onsets[-1]:qrs_peak], height=0.1*np.max(ecg_signal_filtered), distance=0.1*fs)
        if len(p_peak) > 0:
            p_peaks.append(p_peak[-1] + qrs_onsets[-1])
        else:
            p_peaks.append(None)
        # Find the T wave peak
        t_peak, _ = find_peaks(-ecg_signal_filtered[qrs_peak:qrs_offsets[-1]], height=-0.1*np.max(ecg_signal_filtered), distance=0.2*fs)
        if len(t_peak) > 0:
            t_peaks.append(t_peak[0] + qrs_peak)
        else:
            t_peaks.append(None)

    return qrs_peaks, qrs_onsets, qrs_offsets, p_peaks,t_peaks,integrated_signal

In [ ]:
qrs_peaks, qrs_onsets, qrs_offsets, p_peaks,t_peaks,integrated_signal =pan_tompkins(sig3, fs=1000)

In [ ]:
feature_array = []
ecg_signal_filtered=filtered_signal.copy()
for i in range(len(qrs_peaks)):
    feature_vector = []
    # Add QRS complex features
    feature_vector.append(ecg_signal_filtered[qrs_peaks[i]])
    feature_vector.append(integrated_signal[qrs_peaks[i]])
    feature_vector.append(qrs_offsets[i] - qrs_onsets[i])
    feature_vector.append(qrs_onsets[i] - qrs_onsets[0])
    
    if p_peaks[i] is not None:
        feature_vector.append(ecg_signal_filtered[p_peaks[i]])
        feature_vector.append(qrs_peaks[i] - p_peaks[i])
    else:
        feature_vector += [None, None]
    # Add T wave features
    if t_peaks[i] is not None:
        feature_vector.append(ecg_signal_filtered[t_peaks[i]])
        feature_vector.append(t_peaks[i] - qrs_peaks[i])
    else:
        feature_vector += [None, None]
    feature_array.append(feature_vector)

In [ ]:
len(feature_array)

In [ ]:
plt.plot(feature_array[3])

# another try to calculate fiducial features

In [ ]:
import numpy as np
from scipy.signal import butter, filtfilt

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

def pan_tompkins(ecg_signal, fs=1000):
    # Bandpass filter the ECG signal
    ecg_signal_filtered = butter_bandpass_filter(ecg_signal, 2, 50, fs)

    # Differentiate the signal
    diff_signal = np.diff(ecg_signal_filtered)

    # Square the signal
    squared_signal = diff_signal ** 2

    # Integrate the signal
    integrated_signal = np.convolve(squared_signal, np.ones(fs // 8))[:len(ecg_signal)]

    # Find the QRS complex peaks using a threshold-based approach
    qrs_peaks, _ = find_peaks(integrated_signal, height=0.35*np.max(integrated_signal), distance=0.2*fs)

    # Calculate the fiducial features
    qrs_onsets = []
    qrs_offsets = []
    for qrs_peak in qrs_peaks:
        # Find the QRS onset
        for i in range(qrs_peak, 0, -1):
            if integrated_signal[i] < integrated_signal[qrs_peak] * 0.15:
                qrs_onsets.append(i)
                break
        # Find the QRS offset
        for i in range(qrs_peak, len(ecg_signal)):
            if integrated_signal[i] < integrated_signal[qrs_peak] * 0.2:
                qrs_offsets.append(i)
                break

    return qrs_peaks, qrs_onsets, qrs_offsets 

def extract_fiducial_features(ecg_signal, fs=1000):
    # Detect QRS complexes using the Pan-Tompkins algorithm
    qrs_peaks, qrs_onsets, qrs_offsets = pan_tompkins(ecg_signal, fs=fs)

    # Calculate other fiducial features, such as P and T wave peaks, if needed

    return qrs_peaks, qrs_onsets, qrs_offsets


In [ ]:
import scipy.signal as signal

def detect_t_wave_peaks(ecg_signal, qrs_onsets, qrs_offsets, fs=1000):
    t_wave_peaks = []
    for i in range(len(qrs_onsets)):
        qrs_complex = ecg_signal[qrs_onsets[i]:qrs_offsets[i]]
        st_segment = ecg_signal[qrs_offsets[i]:qrs_offsets[i]+int(0.2*fs)]
        st_mean = np.mean(st_segment)
        st_std = np.std(st_segment)
        threshold = st_mean + 0.5 * st_std  # adjust threshold as needed
        t_wave_peak, _ = signal.find_peaks(ecg_signal[qrs_offsets[i]:qrs_offsets[i]+int(0.6*fs)], height=threshold)
        if len(t_wave_peak) > 0:
            t_wave_peak = t_wave_peak[0] + qrs_offsets[i]
            t_wave_peaks.append(t_wave_peak)
    return t_wave_peaks

In [ ]:
import numpy as np

def extract_features(ecg_signal, qrs_peaks, qrs_onsets, qrs_offsets, fs=1000):
    # Calculate heart rate variability (HRV)
    rr_intervals = np.diff(qrs_peaks) / fs
    mean_rr = np.mean(rr_intervals)
    std_rr = np.std(rr_intervals)

    # Calculate QRS duration
    qrs_durations = (qrs_offsets - qrs_onsets) / fs
    mean_qrs_duration = np.mean(qrs_durations)
    std_qrs_duration = np.std(qrs_durations)

    # Calculate QT interval
    t_wave_peaks = detect_t_wave_peaks(ecg_signal, qrs_onsets, qrs_offsets, fs)
    qt_intervals = (t_wave_peaks - qrs_onsets) / fs
    mean_qt_interval = np.mean(qt_intervals)
    std_qt_interval = np.std(qt_intervals)

    # Calculate ST segment deviation
    st_segments = []
    for i in range(len(qrs_peaks)-1):
        start = int((qrs_peaks[i] + qrs_offsets[i])/2)  # mid-point of the QRS complex
        end = start + int(0.2 * fs)  # 0.2 seconds after the mid-point
        st_segments.append( ecg_signal[int(start):int(end)] - np.mean(ecg_signal[int(start-0.1*fs):int(start)]))
  # deviation from baseline
    st_segments_array = np.vstack(st_segments)
    mean_st_deviation = np.mean(st_segments_array)
    std_st_deviation = np.std(st_segments_array)

    return {
        'mean_rr': mean_rr,
        'std_rr': std_rr,
        'mean_qrs_duration': mean_qrs_duration,
        'std_qrs_duration': std_qrs_duration,
        'mean_qt_interval': mean_qt_interval,
        'std_qt_interval': std_qt_interval,
        'mean_st_deviation': mean_st_deviation,
        'std_st_deviation': std_st_deviation,
    }


In [ ]:
qrs_peaks, qrs_onsets, qrs_offsets=extract_fiducial_features(filtered_signal, fs=1000)

res=extract_features(filtered_signal,np.array(qrs_peaks), np.array(qrs_onsets), np.array(qrs_offsets), fs=1000)